In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
S = 1.20         # Spot price (USD per pound)
r = 0.02         # Risk-free interest rate (2%)
d = 0.01         # Storage cost (1%)
y = 0.00         # Convenience yield (assumed 0)
T = 0.5          # Time to maturity (6 months = 0.5 years)
X = 1.25         # Strike price
sigma = 0.25     # Volatility (25%)

# Implement Cost Of Carry Model
- The Cost of Carry Model is used to calculate the fair price of a futures contract.
- It accounts for the spot price, interest rates, storage costs and convenience yield.
- This helps traders identify mispricing and arbitrage opportunities in commodity markets.

In [4]:
# Futures price formula: F = S * exp((r + d - y) * T)
F = S * np.exp((r + d - y) * T)
F.round(2)

np.float64(1.22)

- It means the fair price the client should pay for the futures contract is 1.22 (USD).
- This price includes the current value of the product, plus costs like storage and interest, minus any benefits of holding it now.

# Black-Scholes Model for Futures Option

In [6]:
d1 = (np.log(F / X) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
d2.round(2)

np.float64(-0.23)

- The value -0.23 means the option is less likely to make a profit at expiration.
- It suggests the futures price is expected to stay below the strike price.
- So, the option is probably not worth much right now.

# European call option price on futures

In [7]:
call_price_bs = np.exp(-r * T) * (F * norm.cdf(d1) - X * norm.cdf(d2))
call_price_bs.round(2)

np.float64(0.07)

- The fair value of the European call option is $0.07.
- This means you'd pay 7 cents per unit of the futures contract to buy the option today.

# Monte Carlo Simulation

In [9]:
np.random.seed(42)
num_simulations = 100000

# Simulate futures prices at maturity using GBM
Z = np.random.standard_normal(num_simulations)
ST = F * np.exp(-0.5 * sigma**2 * T + sigma * np.sqrt(T) * Z)

# Calculate payoffs for call option
payoffs = np.maximum(ST - X, 0)

# Discounted average payoff
call_price_mc = np.exp(-r * T) * np.mean(payoffs)

# Display Results

In [10]:
print("Futures Price (Cost of Carry):", round(F, 4))
print("Call Option Price (Black-Scholes):", round(call_price_bs, 4))
print("Call Option Price (Monte Carlo Simulation):", round(call_price_mc, 4))

Futures Price (Cost of Carry): 1.2181
Call Option Price (Black-Scholes): 0.0712
Call Option Price (Monte Carlo Simulation): 0.0714


- $1.2181 → Fair futures price for coffee in 6 months.
- $0.0712 / $0.0714 → Fair price to buy a 6-month option to purchase coffee at $1.25.

## Pipeline Used ChatGtp to combine into something more intepretable

In [11]:
import numpy as np
from scipy.stats import norm

def black_scholes_futures_call(S, r, d, y, T, X, sigma):
    # Step 1: Calculate the fair futures price using Cost of Carry model
    F = S * np.exp((r + d - y) * T)
    
    # Step 2: Compute d1 and d2
    d1 = (np.log(F / X) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Step 3: Calculate call option price
    call_price_bs = np.exp(-r * T) * (F * norm.cdf(d1) - X * norm.cdf(d2))
    
    # Step 4: Interpret results in simpler words
    interpretation = ""
    if d2 < 0:
        interpretation = (
            f"The option is less likely to end in profit since d2 = {d2:.2f} is negative. "
            f"This means the futures price is expected to stay below the strike price (${X}). "
            "So the option is relatively cheap and may not be exercised."
        )
    else:
        interpretation = (
            f"The option has a higher chance of profit since d2 = {d2:.2f} is positive. "
            f"This means the futures price is expected to be above the strike price (${X}) at expiration. "
            "So the option is more valuable and likely to be exercised."
        )
    
    return {
        "Futures Price (F)": round(F, 4),
        "d1": round(d1, 4),
        "d2": round(d2, 4),
        "Call Option Price": round(call_price_bs, 4),
        "Interpretation": interpretation
    }


# Scenario 1

In [12]:
result = black_scholes_futures_call(
    S=1.15,    # Spot price
    r=0.03,    # Risk-free rate
    d=0.01,    # Storage cost
    y=0,       # Convenience yield
    T=0.5,     # Time to maturity in years
    X=1.20,    # Strike price
    sigma=0.25 # Volatility
)

for k, v in result.items():
    print(f"{k}: {v}")

Futures Price (F): 1.1732
d1: -0.0392
d2: -0.216
Call Option Price: 0.0698
Interpretation: The option is less likely to end in profit since d2 = -0.22 is negative. This means the futures price is expected to stay below the strike price ($1.2). So the option is relatively cheap and may not be exercised.
